# Scale Data with Outliers

Adapted from Jason Brownlee. 2020. [Data Preparation for Machine Learning](https://machinelearningmastery.com/data-preparation-for-machine-learning/).


## Overview

Many machine learning algorithms perform better when numerical input variables are scaled to
a standard range. This includes algorithms that use a weighted sum of the input, like linear
regression, and algorithms that use distance measures, like k-nearest neighbors. Standardizing
is a popular scaling technique that subtracts the mean from values and divides by the standard
deviation, transforming the probability distribution for an input variable to a standard Gaussian
(zero mean and unit variance). Standardization can become skewed or biased if the input
variable contains outlier values.
To overcome this, the median and interquartile range can be used when standardizing
numerical input variables, generally referred to as robust scaling.

## Learning Objectives

- Understand why many machine learning algorithms prefer scaled numerical input variables
- Learn robust scaling techniques that use percentiles to scale numerical input variables containing outliers
- Master using `RobustScaler` to scale numerical input variables using median and interquartile range
- Evaluate the impact of different scaling ranges on model performance

### Tasks to complete

- Examine raw dataset distributions
- Apply robust scaling transformation
- Evaluate model performance with different scaling ranges
- Compare results between scaled and unscaled data


## Prerequisites

- A working Python environment
- Basic understanding of Python programming concepts
- Basic understanding of machine learning concepts
- Familiarity with pandas and numpy libraries
- Knowledge of basic statistical concepts


## Get Started

To start, we install required packages and import necessary libraries.


### Install packages

In [ ]:
%pip install matplotlib numpy pandas scikit-learn

### Import libraries

In [ ]:
from matplotlib import pyplot
from numpy import mean, std
from pandas import DataFrame, read_csv
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, RobustScaler

pima_indians_diabetes_csv = "../../Data/pima-indians-diabetes.csv"

## Robust Scaling Data

If there are input
variables that have very large values relative to the other input variables, these large values
can dominate or skew some machine learning algorithms. The result is that the algorithms pay
most of their attention to the large values and ignore the variables with smaller values.

Sometimes an input variable may have outlier values. Outliers can skew a probability distribution and make data scaling using standardization
difficult as the calculated mean and standard deviation will be skewed by the presence of
the outliers. 

Robust standardization or robust data
scaling that calculates the median (50th percentile) and the 25th and
75th percentiles. The values of each variable then have their median subtracted and are divided
by the interquartile range (IQR) which is the difference between the 75th and 25th percentiles


## Diabetes Dataset

The dataset classifies patient data as
either an onset of diabetes within five years or not.

```
Number of Instances: 768
Number of Attributes: 8 plus class 
For Each Attribute: (all numeric-valued)
   1. Number of times pregnant
   2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
   3. Diastolic blood pressure (mm Hg)
   4. Triceps skin fold thickness (mm)
   5. 2-Hour serum insulin (mu U/ml)
   6. Body mass index (weight in kg/(height in m)^2)
   7. Diabetes pedigree function
   8. Age (years)
   9. Class variable (0 or 1)
Missing Attribute Values: Yes
Class Distribution: (class value 1 is interpreted as "tested positive for
   diabetes")
   Class Value  Number of instances
   0            500
   1            268
```

You can learn more about the dataset here:

* Diabetes Dataset File ([pima-indians-diabetes.csv](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv))
* Diabetes Dataset Details ([pima-indians-diabetes.names](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.names))

In [ ]:
# load and summarize the diabetes dataset

# load dataset
dataset = read_csv(pima_indians_diabetes_csv, header=None)
print(dataset.head())

# summarize the shape of the dataset
print(dataset.shape)

# summarize each variable
print(dataset.describe())

This confirms the 8
input variables, one output variable, and 768 rows of data. A statistical summary of the input
variables is provided show that each variable has a very different scale. This makes it a good
dataset for exploring data scaling methods.

We can create a histogram for each input variable. The plots confirm the differing scale
for each input variable and show that the variables have differing scales. Importantly, we can see
some of the distributions show the presence of outliers. The dataset provides a good candidate for using a robust scaler transform to standardize the data in the presence of di ering input
variable scales and outliers.

In [ ]:
# histograms of the variables
fig = dataset.hist(xlabelsize=4, ylabelsize=4)
[x.title.set_size(4) for x in fig.ravel()]

# show the plot
pyplot.show()

Next, let's fit and evaluate a machine learning model on the raw dataset. We will use
a k-nearest neighbor algorithm with default hyperparameters and evaluate it using repeated
stratified k-fold cross-validation.

In [ ]:
# evaluate knn on the raw diabetes dataset

# load dataset
dataset = read_csv(pima_indians_diabetes_csv, header=None)
data = dataset.values

# separate into input and output columns
X, y = data[:, :-1], data[:, -1]

# ensure inputs are floats and output is an integer label
X = X.astype("float32")
y = LabelEncoder().fit_transform(y.astype("str"))

# define and configure the model
model = KNeighborsClassifier()

# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring="accuracy", cv=cv, n_jobs=-1)

# report model performance
print("Accuracy: %.3f (%.3f)" % (mean(n_scores), std(n_scores)))

In this case we can see that the model achieved a mean classi cationfiaccuracy of about
71.7 percent.

Next, let's explore a robust scaling transform of the dataset.

## IQR Robust Scaler Transform

We can apply the robust scaler to the diabetes dataset directly. We will use the default
configuration and scale values to the IQR. First, a **RobustScaler** instance is defined with
default hyperparameters. Once defined, we can call the **fit.transform()** function and pass it
to our dataset to create a robust scale transformed version of our dataset.

In [ ]:
# visualize a robust scaler transform of the diabetes dataset

# load dataset
dataset = read_csv(pima_indians_diabetes_csv, header=None)

# retrieve just the numeric input values
data = dataset.values[:, :-1]

# perform a robust scaler transform of the dataset
trans = RobustScaler()
data = trans.fit_transform(data)

# convert the array back to a dataframe
dataset = DataFrame(data)

# summarize
print(dataset.describe())

We can see that the
distributions have been adjusted. The median values are now zero and the standard deviation
values are now close to 1.0.

In [ ]:
# histograms of the variables
fig = dataset.hist(xlabelsize=4, ylabelsize=4)
[x.title.set_size(4) for x in fig.ravel()]

# show the plot
pyplot.show()

Histogram plots of the variables are created, although the distributions don't look much
different from their original distributions seen in the previous section. We can see that the
center of mass for each distribution is now close to zero.

Next, let's evaluate the same KNN model as the previous section, but in this case on a
robust scaler transform of the dataset.

In [ ]:
# evaluate knn on the diabetes dataset with robust scaler transform

# load dataset
dataset = read_csv(pima_indians_diabetes_csv, header=None)
data = dataset.values

# separate into input and output columns
X, y = data[:, :-1], data[:, -1]

# ensure inputs are floats and output is an integer label
X = X.astype("float32")
y = LabelEncoder().fit_transform(y.astype("str"))

# define the pipeline
trans = RobustScaler()
model = KNeighborsClassifier()
pipeline = Pipeline(steps=[("t", trans), ("m", model)])

# evaluate the pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring="accuracy", cv=cv, n_jobs=-1)

# report pipeline performance
print("Accuracy: %.3f (%.3f)" % (mean(n_scores), std(n_scores)))

We can see that the robust scaler transform results in a lift in
performance from 71.7 percent accuracy without the transform to about 73.4 percent with the
transform.

## Explore Robust Scaler Range

The range used to scale each variable is chosen by default as the IQR is bounded by the 25th
and 75th percentiles. This is specified by the quantile range argument as a tuple. Other
values can be specified and might improve the performance of the model, such as a wider range,
allowing fewer values to be considered outliers, or a more narrow range, allowing more values
to be considered outliers. The example below explores the effect of different definitions of the
range from 1st to the 99th percentiles to 30th to 70th percentiles.

In [ ]:
# explore the scaling range of the robust scaler transform

# get the dataset
def get_dataset():
    # load dataset
    dataset = read_csv(pima_indians_diabetes_csv, header=None)
    data = dataset.values
    # separate into input and output columns
    X, y = data[:, :-1], data[:, -1]
    # ensure inputs are floats and output is an integer label
    X = X.astype("float32")
    y = LabelEncoder().fit_transform(y.astype("str"))
    return X, y


# get a list of models to evaluate
def get_models():
    models = dict()
    for value in [1, 5, 10, 15, 20, 25, 30]:
        # define the pipeline
        trans = RobustScaler(quantile_range=(value, 100 - value))
        model = KNeighborsClassifier()
        models[str(value)] = Pipeline(steps=[("t", trans), ("m", model)])
    return models


# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring="accuracy", cv=cv, n_jobs=-1)
    return scores


# define dataset
X, y = get_dataset()
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X, y)
    results.append(scores)
    names.append(name)
    print(">%s %.3f (%.3f)" % (name, mean(scores), std(scores)))


We can see that ranges such as 10-90 and 15-85 perform better than the default
of 25-75.

In [ ]:
# plot model performance for comparison
pyplot.boxplot(results, tick_labels=names, showmeans=True)
pyplot.show()

Box and whisker plots are created to summarize the classification accuracy scores for each
IQR range. We can see a subtle difference in the distribution and mean accuracy with the larger
ranges of 15-85 vs 25-75. percentiles.

## Conclusion

Robust scaling provides an effective way to standardize numerical variables when outliers are present. Different scaling ranges (like 10-90 or 15-85 percentiles) can outperform the default 25-75 percentile range, showing the importance of testing different scaling parameters.

## Clean up

Remember to shut down your Jupyter Notebook environment and delete any unnecessary files or resources once you've completed the tutorial.